In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Irritable_bowel_syndrome_(IBS)"
cohort = "GSE66824"

# Input paths
in_trait_dir = "../../input/GEO/Irritable_bowel_syndrome_(IBS)"
in_cohort_dir = "../../input/GEO/Irritable_bowel_syndrome_(IBS)/GSE66824"

# Output paths
out_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/GSE66824.csv"
out_gene_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/gene_data/GSE66824.csv"
out_clinical_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/clinical_data/GSE66824.csv"
json_path = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genomic and Clinical Effects Associated with a Relaxation Response Mind-Body Intervention in Patients with Irritable Bowel Syndrome and Inflammatory Bowel Disease"
!Series_summary	"Patients with chronic illnesses such as Irritable Bowel Syndrome (IBS) or Inflammatory Bowel Disease (IBD) often have reduced quality of life. IBS is characterized by abdominal pain/discomfort associated with altered bowel function, such as diarrhea or constipation, without gross structural changes or inflammation [1]; IBD is characterized by gross inflammation in the gastrointestinal (GI) tract which can result in symptoms such as abdominal pain, cramping, diarrhea and bloody stools.  IBS and IBD can profoundly affect quality of life and are influenced by stress and resiliency.The impact of mind-body interventions (MBIs) on IBS and IBD patients has not previously been examined. In this study IBS and  IBD patients were enrolled in a 9-week relaxation response based mind

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Optional, Callable, Dict, Any, List

# 1. Gene Expression Data Availability
# Based on the background information, there is transcriptome analysis using Affymetrix arrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (IBS/IBD):
# From sample characteristics, key 1 and 4 both contain disease state information
trait_row = 1  # Using the first occurrence

# For age:
# No age information is available in the sample characteristics
age_row = None

# For gender:
# No gender information is available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value: str) -> int:
    """Convert trait value to binary (0 for IBD, 1 for IBS)"""
    if pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "IBS" in value:
        return 1  # IBS is our target trait
    elif "IBD" in value:
        return 0  # IBD is the control
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """Convert age value to float"""
    # Not used since age data is not available
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender value to binary (0 for female, 1 for male)"""
    # Not used since gender data is not available
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and record cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only execute if trait data is available
if trait_row is not None:
    # Create a DataFrame from the sample characteristics for clinical data extraction
    sample_char_dict = {0: ['patient: A01', 'patient: A02', 'patient: A03', 'patient: A06', 'patient: A07', 'patient: A10', 'patient: A13', 'patient: A14', 'patient: A15', 'patient: A16', 'patient: A18', 'patient: A20', 'patient: A23', 'patient: A25', 'patient: B01', 'patient: B03', 'patient: B04', 'patient: B06', 'patient: B07', 'patient: B08', 'patient: B09', 'patient: B11', 'patient: B12', 'patient: B13', 'patient: B14', 'patient: B16', 'patient: B20', 'patient: B22', 'patient: B23', 'patient: B24'], 
                     1: ['disease state: IBS', 'disease state: IBD'], 
                     2: ['time point: baseline before intervention', 'time point: after 9 weeks of intervention', 'time point: before intervention'], 
                     3: ['tissue: peripheral blood'], 
                     4: ['disease state: IBS', 'disease state: IBD'], 
                     5: ['time point: baseline before intervention', 'time point: after 9 weeks of intervention', 'time point: before intervention'], 
                     6: ['tissue: peripheral blood']}
    
    clinical_data = pd.DataFrame()
    
    # Add each row from the sample characteristics dictionary as a column
    for key, values in sample_char_dict.items():
        clinical_data[key] = values if len(values) == len(clinical_data) else values + [None] * (len(clinical_data) - len(values))
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(selected_clinical_df)
    print(f"Preview of selected clinical features: {preview}")
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features: {0: [nan], 1: [0.0], 2: [nan], 3: [nan], 4: [0.0], 5: [nan], 6: [nan]}
Clinical data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/clinical_data/GSE66824.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 69
Header line: "ID_REF"	"GSM1632703"	"GSM1632704"	"GSM1632705"	"GSM1632706"	"GSM1632707"	"GSM1632708"	"GSM1632709"	"GSM1632710"	"GSM1632711"	"GSM1632712"	"GSM1632713"	"GSM1632714"	"GSM1632715"	"GSM1632716"	"GSM1632717"	"GSM1632718"	"GSM1632719"	"GSM1632720"	"GSM1632721"	"GSM1632722"	"GSM1632723"	"GSM1632724"	"GSM1632725"	"GSM1632726"	"GSM1632727"	"GSM1632728"	"GSM1632729"	"GSM1632730"	"GSM1632731"	"GSM1632732"	"GSM1632733"	"GSM1632734"	"GSM1632735"	"GSM1632736"	"GSM1632737"	"GSM1632738"	"GSM1632739"	"GSM1632740"	"GSM1632741"	"GSM1632742"	"GSM1632743"	"GSM1632744"	"GSM1632745"	"GSM1632746"	"GSM1632747"	"GSM1632748"	"GSM1632749"	"GSM1632750"	"GSM1632751"	"GSM1632752"	"GSM1632753"	"GSM1632754"	"GSM1632755"	"GSM1632756"	"GSM1632757"	"GSM1632758"	"GSM1632759"	"GSM1632760"	"GSM1632761"	"GSM1632762"	"GSM1632763"	"GSM1632764"	"GSM1632765"	"GSM1632766"	"GSM1632767"	"GSM1632768"
First data line: "1007_PM_s_at"	5.84456	6.18582	5.95226	6.27915	5.99674	5.9074	6.1111	6.127

Index(['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at',
       '1294_PM_at', '1316_PM_at', '1320_PM_at', '1405_PM_i_at', '1431_PM_at',
       '1438_PM_at', '1487_PM_at', '1494_PM_f_at', '1552256_PM_a_at',
       '1552257_PM_a_at', '1552258_PM_at', '1552261_PM_at', '1552263_PM_at',
       '1552264_PM_a_at', '1552266_PM_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers shown (like "1007_PM_s_at", "1053_PM_at", etc.), 
# these appear to be Affymetrix probe IDs from a microarray platform.
# These are not standard human gene symbols and will need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the column names for gene identifiers and gene symbols
# From the previous output, we can see:
# - "ID" column contains the probe identifiers matching those in the gene expression data
# - "Gene Symbol" column contains the human gene symbols we need to map to

# 2. Get a gene mapping dataframe
# Extract the two relevant columns from the gene annotation dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col="ID", gene_col="Gene Symbol")

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This divides expression values for probes mapping to multiple genes and sums values for the same gene
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Display information about the resulting gene expression data
print(f"Number of genes after mapping: {len(gene_data)}")
print(f"Sample of gene expression data (first 5 genes, first 3 samples):")
print(gene_data.iloc[:5, :3])

# Save the gene expression data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Number of genes after mapping: 18989
Sample of gene expression data (first 5 genes, first 3 samples):
       GSM1632703  GSM1632704  GSM1632705
Gene                                     
A1BG      3.71206     4.07512     4.23269
A1CF      3.78140     3.36702     2.83565
A2BP1    17.62408    13.62811    13.40417
A2LD1     5.86426     7.02061     7.64654
A2M      10.69333    10.46429    10.85364


Gene expression data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/gene_data/GSE66824.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Load the normalized gene data 
gene_data = pd.read_csv(out_gene_data_file, index_col=0)

# 2. Re-extract clinical features from the SOFT file to get proper clinical data
# Use the actual clinical data from the matrix file properly
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# 3. Create a correct clinical features dataframe
# First inspect what's in the clinical data
clinical_data_dict = get_unique_values_by_row(clinical_data)
print("Clinical data structure:")
print(clinical_data_dict)

# Based on the sample characteristics dictionary shown previously, 
# extract and process clinical features
selected_clinical_df = pd.DataFrame()

# Process disease state row manually to ensure correct mapping
disease_row = clinical_data.iloc[trait_row]
samples = [col for col in disease_row.index if col != "!Sample_geo_accession"]
trait_values = []

for sample in samples:
    value = disease_row[sample]
    if pd.isna(value):
        trait_values.append(None)
    else:
        if ":" in value:
            value = value.split(":", 1)[1].strip()
        
        if "IBS" in value:
            trait_values.append(1)  # IBS is our target trait
        elif "IBD" in value:
            trait_values.append(0)  # IBD is the control
        else:
            trait_values.append(None)

# Create dataframe with processed values
selected_clinical_df[trait] = trait_values
selected_clinical_df.index = samples

# Save the corrected clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Corrected clinical data saved to {out_clinical_data_file}")

# 4. Link the clinical and genetic data
linked_data = pd.DataFrame()
# Transpose gene data to have samples as rows and genes as columns
gene_data_t = gene_data.T
# Verify alignment of sample IDs between clinical and gene data
common_samples = list(set(selected_clinical_df.index) & set(gene_data_t.index))
if common_samples:
    gene_data_filtered = gene_data_t.loc[common_samples]
    clinical_data_filtered = selected_clinical_df.loc[common_samples]
    # Join the data
    linked_data = pd.concat([clinical_data_filtered, gene_data_filtered], axis=1)
    print(f"Linked data shape: {linked_data.shape}")
else:
    # Alternative linking approach if sample IDs don't directly match
    print("No common sample IDs found. Attempting alternative linking...")
    # The GSM ids in gene data columns may correspond to the sample IDs
    clinical_data_reset = selected_clinical_df.reset_index()
    clinical_data_reset.columns = ["Sample"] + list(clinical_data_reset.columns[1:])
    gene_data_cols = list(gene_data.columns)
    
    # Create merged dataframe
    data_dict = {trait: []}
    # Add trait values
    for col in gene_data_cols:
        sample_idx = clinical_data_reset.index[clinical_data_reset["Sample"] == col] if "Sample" in clinical_data_reset.columns else []
        if len(sample_idx) > 0:
            data_dict[trait].append(clinical_data_reset.loc[sample_idx[0], trait])
        else:
            data_dict[trait].append(None)
    
    # Add gene expression values
    for gene in gene_data.index:
        data_dict[gene] = list(gene_data.loc[gene])
    
    linked_data = pd.DataFrame(data_dict, index=gene_data_cols)
    print(f"Alternative linked data shape: {linked_data.shape}")

# 5. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Data after handling missing values: {linked_data.shape}")

# 6. Determine whether the trait and some demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 7. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from patients with IBS and IBD, examining effects of relaxation response mind-body intervention."
)

# 8. If the linked data is usable, save it as a CSV file
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Clinical data structure:
{0: ['patient: A01', 'patient: A02', 'patient: A03', 'patient: A06', 'patient: A07', 'patient: A10', 'patient: A13', 'patient: A14', 'patient: A15', 'patient: A16', 'patient: A18', 'patient: A20', 'patient: A23', 'patient: A25', 'patient: B01', 'patient: B03', 'patient: B04', 'patient: B06', 'patient: B07', 'patient: B08', 'patient: B09', 'patient: B11', 'patient: B12', 'patient: B13', 'patient: B14', 'patient: B16', 'patient: B20', 'patient: B22', 'patient: B23', 'patient: B24'], 1: ['disease state: IBS', 'disease state: IBD'], 2: ['time point: baseline before intervention', 'time point: after 9 weeks of intervention', 'time point: before intervention'], 3: ['tissue: peripheral blood'], 4: ['disease state: IBS', 'disease state: IBD'], 5: ['time point: baseline before intervention', 'time point: after 9 weeks of intervention', 'time point: before intervention'], 6: ['tissue: peripheral blood']}
Corrected clinical data saved to ../../output/preprocess/Irritable_

Data after handling missing values: (66, 18990)
For the feature 'Irritable_bowel_syndrome_(IBS)', the least common label is '1' with 28 occurrences. This represents 42.42% of the dataset.
The distribution of the feature 'Irritable_bowel_syndrome_(IBS)' in this dataset is fine.



Linked data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/GSE66824.csv
